# Text Generation with LSTM (Keras)

##### Step 1: Install Required Libraries

In [1]:
!pip install tensorflow nltk --quiet

##### Step 2: Import Libraries

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


##### Step 3: Prepare Dataset (Shakespeare - Hamlet)

In [6]:

text = gutenberg.raw('shakespeare-hamlet.txt').lower()
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create sequences
input_sequences = []
words = text.split()
for i in range(10, len(words)):
    seq = words[i-10:i+1]
    line = tokenizer.texts_to_sequences([' '.join(seq)])[0]
    input_sequences.append(line)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=11, padding='pre'))
X = input_sequences[:, :-1]
y = tf.keras.utils.to_categorical(input_sequences[:, -1], num_classes=total_words)


##### Step 4: Build GPU-Compatible LSTM Model

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=10))
model.add(LSTM(150))  # ✅ This will use CuDNN with T4 GPU
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

##### Step 5: Train Model

In [8]:
model.fit(X, y, epochs=5, verbose=1)

Epoch 1/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.0318 - loss: 7.0111
Epoch 2/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 35s 38ms/step - accuracy: 0.0449 - loss: 6.3570
Epoch 3/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 43s 40ms/step - accuracy: 0.0597 - loss: 6.1350
Epoch 4/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 36s 39ms/step - accuracy: 0.0785 - loss: 5.8124
Epoch 5/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 36s 39ms/step - accuracy: 0.0967 - loss: 5.4457


##### Step 6: Text Generation Function

In [9]:
def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=10, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word = tokenizer.index_word.get(np.argmax(predicted), '')
        seed_text += ' ' + predicted_word
    return seed_text

# LSTM-Specific Prompts

LSTM-Specific Prompts
(Since LSTM models are trained on Hamlet, keep the tone poetic or dramatic.)

"To be or not to be, that is"

"O, what a noble mind is here"

"Thou art more lovely and more"

"The king hath spoken of"

"Dost thou hear the winds of"

"Speak the truth, and thou shalt"

"My lord, the hour is late, and"

"Fear not the shadows, for they"

"He comes bearing news of"

"By my troth, I saw the ghost of"



##### Step 7: User Input

In [10]:
user_input = input("👉 Enter a topic or starting phrase: ")
print("\n📝 Generated Paragraph:\n")
print(generate_text(user_input))

👉 Enter a topic or starting phrase: "To be or not to be, that is"

📝 Generated Paragraph:

"To be or not to be, that is" the king and the king is the king ham i am not to the king ham i am not to the king ham i am not to the king ham i am not to the king ham i am not to the king ham i am not to the king
